# WF

In [34]:
%reload_ext autoreload
%autoreload 2
import colabexts
from colabexts.jcommon import *
import xmltodict
import colabexts.utils as colabexts_utils


import os, sys, datetime, re, json, importlib
from collections import defaultdict
from sys import modules
from IPython.display import HTML, Javascript
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

%matplotlib inline
pd.set_option('display.max_colwidth', 40)
pd.set_option('display.max_rows', 16)

import logging
for handler in logging.root.handlers[:]: logging.root.removeHandler(handler)
logging.basicConfig( level=logging.INFO,
        format='%(asctime)s %(name)s %(levelname)s: %(message)s',
        #handlers=[ logging.FileHandler("/tmp/stream.log"), logging.StreamHandler()],
        handlers=[ logging.StreamHandler()],
)
logger = logging.getLogger( "geoapp" )
logger.info( f"Testing logging" )

2024-07-12 15:44:23,672 geoapp INFO: Testing logging


# WF_services

In [2]:
%%writefile ../wf_services.py
#!/usr/bin/env python 

#*** DO NOT EDIT - GENERATED FROM tseries/notebooks/tseries_services.ipynb ****

import sys, os,  glob, logging, datetime, json
from  mangorest.mango import webapi
from colabexts import utils as colabexts_utils
import pandas as pd
import xmltodict
import colabexts.utils as colabexts_utils

MBASE  = os.environ.get( "WF_BASE", os.path.expanduser("~/data/workflows/"))
logger = logging.getLogger( "geoapp" )

#---------------------------------------------------------------------------------    
@webapi("/wf/save_wf")
def save_wf( name, jsn, xml, **kwargs):
    name = name or "default"
    if (name.endswith(".json")):  name = name[:-5]
    if (name.endswith(".xml")):   name = name[:-4]
        
    if not os.path.exists(MBASE ):
        os.makedirs(MBASE)
    
    with open(f"{MBASE}/{name}.xml",  "w") as f:   f.write(xml.strip())
    if ( len(jsn) < 30 ):
        jsn = xmltodict.parse(xml,attr_prefix='', cdata_key='#text')
        jsn = json.dumps(jsn, indent=4)

    with open(f"{MBASE}/{name}.json", "w") as f:   f.write(jsn.strip())

    return f"Saved "    
#---------------------------------------------------------------------------------    
@webapi("/wf/get_wf")
def get_wf( name, **kwargs):
    if (name.endswith(".json")):  name = name[:-5]
    if (name.endswith(".xml")):   name = name[:-4]

    xml = open(f"{MBASE}/{name}.xml",  "r").read()
    ret = dict(json="", xml=xml)

    return ret  
#------------------------------------------------------------------------------
@webapi("/wf/delete_wf")
def delete_wf(name="", **kwargs):
    if (name.endswith(".json")):  name = name[:-5]
    if (name.endswith(".xml")):   name = name[:-4]

    logger.info(f"DELETE {name}")
    
    dst = f"{MBASE}/{name}.xml"
    if ( os.path.exists(dst) ):  os.remove(dst)

    dst = f"{MBASE}/{name}.json"
    if ( os.path.exists(dst) ):  os.remove(dst)

    return f"Deleted {dst}."
#------------------------------------------------------------------------------
@webapi("/wf/getall_wfs")
def getAllWorkflows( nrows=10000, patt='*.xml', **kwargs):
    
    files = glob.glob(f"{MBASE}{patt}")
    files.sort(key=os.path.getmtime, reverse=True)

    ret = {
        "name": "workflows",
        "columns": ["name"],
        "values": [[f[len(MBASE):]] for f in files][0:nrows]
    }
    return ret
#---------------------------------------------------------------------------------    
@webapi("/wf/run_wf")
def run_wf( name, **kwargs):
    if (name.endswith(".json")):  name = name[:-5]
    if (name.endswith(".xml")):   name = name[:-4]

    wfdef  = f"/opt/data/data/workflows/{name}.xml"
    buf=open(wfdef,"r").read()
    wf = xmltodict.parse(buf,attr_prefix='', cdata_key='#text')


    wfsh="#Execute\n"
    for c in wf['mxGraphModel']['root']['mxCell']:
        w = colabexts_utils.Map(c)
        s = w.text or ""
        wfsh += f"#Processing ID: {w.id}, Value: {w.value} => Cmd: {s}\n"
        if (not s):
            continue
        wfsh += f"{s}\n"

    open("/tmp/run.sh","w").write(wfsh)

    return f"Running /tmp/run.sh {wfsh} "

Overwriting ../wf_services.py


In [118]:
name='sada'
wfdef  = f"/opt/data/data/workflows/{name}.xml"
buf=open(wfdef,"r").read()
wf = xmltodict.parse(buf,attr_prefix='', cdata_key='#text')


#==================================================================================
def startRunning(w):
    if (w.state == 'running'):
        print("CYCLE DETECTED")
        return ""
    w.state = 'runnning'
    ret=f'''#=> ID: {w.id}, Value: {w.value} => Config: {w.config}
echo "RUNNING: " '{w.config.get('cmd','')}'
{w.config.get('cmd','')}
#semail.py -t "sada@geospaces.org" -m "Step {w.id}, Value: {w.value}  COMPLETE"
'''
    #print(ret)
    for e in w.edges or []:
        if ( e.target != w):
            ret += startRunning(e.target)
    return ret

#==================================================================================
nodes = {}
for c in wf['mxGraphModel']['root']['mxCell']:
    w = colabexts_utils.Map(c)
    nodes[w.id] = w
    w.state = 'init'
    cfg = w.config or "{}"
    try:
        cfg = re.subn('//.*\n', '\n', cfg)[0]
        cfg=colabexts_utils.parsej(cfg)
        w.config = cfg
    except:
        w.config = {}
        pass
    if (not w.edge):
        w['edges']  = []


startNodes = []
for k,w in nodes.items():
    if (w.edge):
       w['source']  = nodes[w['source']]
       w['target']  = nodes[w['target']]
       w['source']['edges'].append(w)
       w['target']['edges'].append(w)
    else:
        typ = w['config'].get("type", "")
        if ( typ == 'start'):
            startNodes.append(w)

wfsh="#Execute:\n"
if (len(startNodes) <=0):
    print("NO START NODES")
else:
    i = 0
    for w in startNodes:
        ret = startRunning(w)

print(wfsh + ret)
open("/tmp/wf.sh", "w").write(wfsh +ret)

#Execute:
#=> ID: 13, Value: S => Config: {'type': 'start', 'run': '[2,3]', 'nrun': 1, 'cmd': 'echo `date` > a.txt'}
echo "RUNNING: " 'echo `date` > a.txt'
echo `date` > a.txt
#semail.py -t "sada@geospaces.org" -m "Step 13, Value: S  COMPLETE"
#=> ID: 2, Value: Start => Config: {'type': 'na|process|start', 'run': '[1,4]', 'nrun': 1, 'cmd': 'echo -- `date` >> a.txt'}
echo "RUNNING: " 'echo -- `date` >> a.txt'
echo -- `date` >> a.txt
#semail.py -t "sada@geospaces.org" -m "Step 2, Value: Start  COMPLETE"
#=> ID: 11, Value: System Admin => Config: {'type': 'na|process|start', 'run': '[10,20]', 'nrun': 1}
echo "RUNNING: " ''

#semail.py -t "sada@geospaces.org" -m "Step 11, Value: System Admin  COMPLETE"
#=> ID: 5, Value: Send Email! OK => Config: {}
echo "RUNNING: " ''

#semail.py -t "sada@geospaces.org" -m "Step 5, Value: Send Email! OK  COMPLETE"
#=> ID: 4, Value: Build Model! => Config: {'type': 'na|process|start', 'run': '[4,5]', 'nrun': 1, 'cmd': 'echo BUILD MODEL >> a.txt'}
echo "RUNN

1557

In [6]:
xml = '''
    '<mxGraphModel><root><mxCell id="0"/><mxCell id="1" parent="0"/>'+
    '<mxCell id="2" value="Claim Handling Process" style="swimlane" vertex="1" parent="1"><mxGeometry x="1" width="850" height="400" as="geometry"/></mxCell>'+
    '<mxCell id="3" value="Claim Manager" style="swimlane" vertex="1" parent="2"><mxGeometry x="30" width="820" height="200" as="geometry"/></mxCell>'+
    '<mxCell id="5" value="" style="start" vertex="1" parent="3"><mxGeometry x="40" y="85" width="30" height="30" as="geometry"/></mxCell>'+
    '<mxCell id="AuthorizeClaim" value="Authorize&#xa;Claim" vertex="1" parent="3"><mxGeometry x="90" y="80" width="100" height="40" as="geometry"/></mxCell>'+
    '<mxCell id="6" value="X" style="step" vertex="1" parent="3"><mxGeometry x="210" y="85" width="30" height="30" as="geometry"/></mxCell>'+
    '<mxCell id="ApproveClaim" value="Approve&#xa;Claim" vertex="1" parent="3"><mxGeometry x="260" y="80" width="100" height="40" as="geometry"/></mxCell>'+
    '<mxCell id="7" value="X" style="step" vertex="1" parent="3"><mxGeometry x="380" y="85" width="30" height="30" as="geometry"/></mxCell>'+
    '<mxCell id="8" value="" edge="1" parent="3" source="5" target="AuthorizeClaim"><mxGeometry relative="1" as="geometry"/></mxCell>'+
    '<mxCell id="9" value="" edge="1" parent="3" source="AuthorizeClaim" target="6"><mxGeometry relative="1" as="geometry"/></mxCell>'+
    '<mxCell id="10" value="" edge="1" parent="3" source="6" target="ApproveClaim"><mxGeometry relative="1" as="geometry"/></mxCell>'+
    '<mxCell id="11" value="" edge="1" parent="3" source="ApproveClaim" target="7"><mxGeometry relative="1" as="geometry"/></mxCell>'+
    '<mxCell id="12" value="" edge="1" parent="3" source="7" target="AuthorizeClaim"><mxGeometry relative="1" as="geometry"><Array as="points"><mxPoint x="140" y="40"/></Array></mxGeometry></mxCell>'+
    '<mxCell id="ReviewClaim" value="Review&#xa;Claim" vertex="1" parent="3"><mxGeometry x="480" y="80" width="100" height="40" as="geometry"/></mxCell>'+
    '<mxCell id="22" value="X" style="step" vertex="1" parent="3"><mxGeometry x="600" y="85" width="30" height="30" as="geometry"/></mxCell>'+
    '<mxCell id="23" value="" edge="1" parent="3" source="ReviewClaim" target="22"><mxGeometry relative="1" as="geometry"/></mxCell>'+
    '<mxCell id="ApproveReviewedClaim" value="Approve Rev.&#xa;Claim" vertex="1" parent="3"><mxGeometry x="650" y="80" width="100" height="40" as="geometry"/></mxCell>'+
    '<mxCell id="26" value="" edge="1" parent="3" source="22" target="ApproveReviewedClaim"><mxGeometry relative="1" as="geometry"/></mxCell>'+
    '<mxCell id="27" value="X" style="step" vertex="1" parent="3"><mxGeometry x="770" y="85" width="30" height="30" as="geometry"/></mxCell>'+
    '<mxCell id="28" value="" edge="1" target="27" parent="3" source="ApproveReviewedClaim"><mxGeometry relative="1" as="geometry"><mxPoint x="740" y="100" as="sourcePoint"/><mxPoint x="760" y="100" as="targetPoint"/></mxGeometry></mxCell>'+
    '<mxCell id="32" value="" edge="1" parent="3" source="27" target="ReviewClaim"><mxGeometry relative="1" as="geometry"><Array as="points"><mxPoint x="665" y="160"/></Array></mxGeometry></mxCell>'+
    '<mxCell id="4" value="Accountant" style="swimlane" vertex="1" parent="2"><mxGeometry x="30" y="200" width="820" height="200" as="geometry"/></mxCell>'+
    '<mxCell id="EnterAccountingData" value="Enter&#xa;Data" vertex="1" parent="4"><mxGeometry x="430" y="80" width="100" height="40" as="geometry"/></mxCell>'+
    '<mxCell id="14" value="X" style="step" vertex="1" parent="4"><mxGeometry x="550" y="85" width="30" height="30" as="geometry"/></mxCell>'+
    '<mxCell id="15" value="" edge="1" parent="4" source="EnterAccountingData" target="14"><mxGeometry relative="1" as="geometry"/></mxCell>'+
    '<mxCell id="CheckAccountingData" value="Check&#xa;Data" vertex="1" parent="4"><mxGeometry x="600" y="80" width="100" height="40" as="geometry"/></mxCell>'+
    '<mxCell id="16" value="" edge="1" parent="4" source="14" target="CheckAccountingData"><mxGeometry relative="1" as="geometry"/></mxCell>'+
    '<mxCell id="17" value="X" style="step" vertex="1" parent="4"><mxGeometry x="720" y="85" width="30" height="30" as="geometry"/></mxCell>'+
    '<mxCell id="18" value="" edge="1" parent="4" source="CheckAccountingData" target="17"><mxGeometry relative="1" as="geometry"/></mxCell>'+
    '<mxCell id="19" value="" style="end" vertex="1" parent="4"><mxGeometry x="770" y="85" width="30" height="30" as="geometry"/></mxCell>'+
    '<mxCell id="20" value="" edge="1" parent="4" source="17" target="19"><mxGeometry relative="1" as="geometry"/></mxCell>'+
    '<mxCell id="31" value="" edge="1" parent="4" source="17" target="EnterAccountingData"><mxGeometry relative="1" as="geometry"><Array as="points"><mxPoint x="625" y="160"/></Array></mxGeometry></mxCell>'+
    '<mxCell id="13" value="" edge="1" parent="2" source="7" target="EnterAccountingData"><mxGeometry relative="1" as="geometry"/></mxCell>'+
    '<mxCell id="24" value="" edge="1" parent="2" source="14" target="ReviewClaim" style="edgeStyle=none"><mxGeometry relative="1" as="geometry"><Array as="points"><mxPoint x="595" y="180"/><mxPoint x="480" y="180"/><mxPoint x="480" y="100"/></Array></mxGeometry></mxCell>'+
    '<mxCell id="29" value="" edge="1" parent="2" source="22" target="EnterAccountingData"><mxGeometry relative="1" as="geometry"><Array as="points"><mxPoint x="469" y="40"/></Array></mxGeometry></mxCell>'+
    '<mxCell id="30" value="" edge="1" parent="2" source="27" target="EnterAccountingData"><mxGeometry relative="1" as="geometry"><Array as="points"><mxPoint x="469" y="40"/></Array></mxGeometry></mxCell>'+
    '<mxCell id="33" value="" edge="1" parent="2" source="6" target="EnterAccountingData"><mxGeometry relative="1" as="geometry"><Array as="points"><mxPoint x="255" y="200"/></Array></mxGeometry></mxCell>'+
    '</root></mxGraphModel>'
'''

In [9]:
x1=xml.replace("'+", "")
x2=x1.replace("'", "")
print(x2)


    <mxGraphModel><root><mxCell id="0"/><mxCell id="1" parent="0"/>
    <mxCell id="2" value="Claim Handling Process" style="swimlane" vertex="1" parent="1"><mxGeometry x="1" width="850" height="400" as="geometry"/></mxCell>
    <mxCell id="3" value="Claim Manager" style="swimlane" vertex="1" parent="2"><mxGeometry x="30" width="820" height="200" as="geometry"/></mxCell>
    <mxCell id="5" value="" style="start" vertex="1" parent="3"><mxGeometry x="40" y="85" width="30" height="30" as="geometry"/></mxCell>
    <mxCell id="AuthorizeClaim" value="Authorize&#xa;Claim" vertex="1" parent="3"><mxGeometry x="90" y="80" width="100" height="40" as="geometry"/></mxCell>
    <mxCell id="6" value="X" style="step" vertex="1" parent="3"><mxGeometry x="210" y="85" width="30" height="30" as="geometry"/></mxCell>
    <mxCell id="ApproveClaim" value="Approve&#xa;Claim" vertex="1" parent="3"><mxGeometry x="260" y="80" width="100" height="40" as="geometry"/></mxCell>
    <mxCell id="7" value="X" style="s